# Persiapan Library dan Dataset
Kita pakai PyTorch untuk implementasi. Dataset MNIST (gambar digit 0-9 hitam putih 28x28 pixel).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np
import os

# Hyperparameter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 0.0002  # Learning rate
batch_size = 128
image_size = 28 * 28  # Untuk MNIST, flatten jadi 784
hidden_size = 256
latent_size = 100  # Ukuran noise input untuk Generator
num_epochs = 50  # Bisa tambah kalau mau hasil lebih baik
beta1 = 0.5  # Untuk Adam optimizer

# Transform dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize ke [-1,1] untuk tanh activation
])

# Load MNIST dataset
dataset = datasets.MNIST(root="dataset/", train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Buat folder untuk simpan gambar dan model
os.makedirs("generated_images", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Arsitektur Generator
Generator: Input noise (latent_size=100), output gambar flatten (784). Pakai fully connected layers.

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size * 2),
            nn.ReLU(),
            nn.Linear(hidden_size * 2, hidden_size * 4),
            nn.ReLU(),
            nn.Linear(hidden_size * 4, image_size),
            nn.Tanh()  # Output [-1,1]
        )
    
    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)  # Reshape ke gambar 1x28x28

# Inisialisasi Generator
generator = Generator().to(device)

# Arsitektur Discriminator
Discriminator: Input gambar (flatten 784), output probabilitas (0-1) apakah real atau fake.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size, hidden_size * 4),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size * 4, hidden_size * 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()  # Output probabilitas
        )
    
    def forward(self, img):
        return self.model(img.view(-1, image_size))  # Flatten input

# Inisialisasi Discriminator
discriminator = Discriminator().to(device)

# Training GAN
Loss: Binary Cross Entropy. Optimizer: Adam.
Kita train Discriminator dulu (dengan real dan fake), lalu Generator (dengan feedback dari Discriminator).

In [ ]:
# Loss dan Optimizer
criterion = nn.BCELoss()
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

# List untuk track loss
g_losses = []
d_losses = []

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size_real = real_images.size(0)
        
        # Train Discriminator
        optimizerD.zero_grad()
        
        # Real images
        real_labels = torch.ones(batch_size_real, 1).to(device)
        output_real = discriminator(real_images)
        lossD_real = criterion(output_real, real_labels)
        
        # Fake images
        z = torch.randn(batch_size_real, latent_size).to(device)  # Noise
        fake_images = generator(z)
        fake_labels = torch.zeros(batch_size_real, 1).to(device)
        output_fake = discriminator(fake_images.detach())
        lossD_fake = criterion(output_fake, fake_labels)
        
        # Total D loss
        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimizerD.step()
        
        # Train Generator
        optimizerG.zero_grad()
        output_fake_forG = discriminator(fake_images)  # Re-use fake images
        lossG = criterion(output_fake_forG, real_labels)  # Ingin D anggap fake sebagai real
        lossG.backward()
        optimizerG.step()
    
    # Simpan loss
    g_losses.append(lossG.item())
    d_losses.append(lossD.item())
    
    # Generate dan simpan gambar tiap 10 epoch
    if (epoch + 1) % 10 == 0 or epoch == 0:
        with torch.no_grad():
            fake = generator(torch.randn(64, latent_size).to(device)).detach().cpu()
            grid = vutils.make_grid(fake, padding=2, normalize=True)
            plt.imshow(np.transpose(grid, (1,2,0)))
            plt.title(f"Epoch {epoch+1}")
            plt.show()
            vutils.save_image(grid, f"generated_images/epoch_{epoch+1}.png")
    
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

# Simpan model
torch.save(generator.state_dict(), "models/generator.pth")
torch.save(discriminator.state_dict(), "models/discriminator.pth")

# Visualisasi Loss Selama Training

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(g_losses, label="Generator Loss")
plt.plot(d_losses, label="Discriminator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("GAN Loss During Training")
plt.show()

# Perbedaan Hasil Antara Epoch Awal dan Akhir
Di epoch awal (misal epoch 1-10), gambar hasil generator biasanya berupa noise acak atau bentuk samar-samar, karena model belum belajar pola digit MNIST dengan baik. Loss G tinggi, D mudah bedain fake.

Di epoch akhir (misal epoch 40-50), gambar lebih jelas menyerupai digit asli (0-9), meski mungkin masih ada artifact. Loss stabil, menandakan equilibrium antara G dan D. Ini menunjukkan GAN berhasil belajar distribusi data MNIST.